In [2]:
import sys
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX_, trainY_), (testX, testY) = cifar10.load_data()
	trainX = trainX_[:45000]
	trainY = trainY_[:45000]
	valX = trainX_[45000:]
	valY = trainY_[45000:]
	# one hot encode target values
	trainY = to_categorical(trainY)
	valY = to_categorical(valY)
	testY = to_categorical(testY)
	return trainX, trainY, valX, valY, testX, testY

In [4]:
# load train and test dataset
def load_dataset():
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    np.random.seed(0)
    val_idx = np.random.choice(range(trainX.shape[0]),5000,replace=False)
    valX = trainX[val_idx]
    valY = trainY[val_idx]

    trainX = np.delete(trainX,val_idx,0)
    trainY = np.delete(trainY,val_idx,0)

    trainY = to_categorical(trainY)
    valY = to_categorical(valY)
    testY = to_categorical(testY)
    return trainX, trainY, valX, valY, testX, testY

In [5]:
# scale pixels
def prep_pixels(train, val, test, standardize_data):
    # convert from integers to floats
    train_norm = train.astype('float32')
    val_norm = val.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    if standardize_data == True:
        mean = np.mean(trainX, 0)
        std = np.std(trainX, 0)
        train_stand = (train_norm - mean) / std
        val_stand = (val_norm - mean) / std
        test_stand = (test_norm - mean) / std
        return train_stand, val_stand, test_stand
    else:
        train_norm = train_norm / 255.0
        val_norm = val_norm / 255.0
        test_norm = test_norm / 255.0
        return train_norm, val_norm, test_norm

In [6]:
def loadData(reduce_data, standardize_data):
    trainX, trainY, valX, valY, testX, testY = load_dataset()
    trainX, valX, testX = prep_pixels(trainX, valX, testX, standardize_data)
    if reduce_data==True:
        return trainX[:10000], trainY[:10000], valX, valY, testX, testY
    else:
        return trainX, trainY, valX, valY, testX, testY

In [7]:
trainX, trainY, valX, valY, testX, testY = loadData(reduce_data=True)
print("trainX shape: ", trainX.shape)
print("trainY shape: ", trainY.shape)
print("valX shape: ", valX.shape)
print("valY shape: ", valY.shape)
print("testX shape: ", testX.shape)
print("testY shape: ", testY.shape)

TypeError: loadData() missing 1 required positional argument: 'standardize_data'

In [ ]:
def scheduler(n_epochs, lr):
    if n_epochs < 10:    
        return lr
    else:
        return lr * math.exp(-0.1)

In [ ]:
def define_model(lr, optimizer, dropout_rate, lamda):

	if optimizer == SGD:
		opt = SGD(lr, momentum = 0.9)
	elif optimizer == Adam:
		opt = Adam(lr)

	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda), input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(lamda)))
	model.add(BatchNormalization())
	model.add(Dropout(dropout_rate))
	model.add(Dense(10, activation='softmax'))
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	return model

In [ ]:
def summarize_diagnostics(history):
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='val')
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='val')
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.close()

In [ ]:
def runNetwork(n_epochs, n_batch, lr=0.001, optimizer=SGD, dropout_rate=0, lamda=0, augmentation=False, standardize_data=False, reduce_data=False):
	trainX, trainY, valX, valY, testX, testY = loadData(reduce_data, standardize_data)

	if augmentation == True:
		model = define_model(lr, optimizer, dropout_rate, lamda)
		datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
		it_train = datagen.flow(trainX, trainY, batch_size=n_batch)
		steps = int(trainX.shape[0] / n_batch)
		callback = LearningRateScheduler(scheduler)
		history = model.fit(it_train, steps_per_epoch=steps, epochs=n_epochs, callbacks=[callback], validation_data=(valX, valY), verbose=1)
	else:
		model = define_model(lr, optimizer, dropout_rate, lamda)
		history = model.fit(trainX, trainY, epochs=n_epochs, batch_size=n_batch, validation_data=(testX, testY), verbose=1)

	_, acc = model.evaluate(valX, valY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	summarize_diagnostics(history)

	return model, history

In [ ]:
model, history = runNetwork(n_epochs=1, n_batch=64, lr=.001, optimizer=Adam, dropout_rate=0.2, lamda=.005, augmentation=True, standardize_data=True, reduce_data=True)